In [1]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker
import matplotlib.colors
import scipy.stats
import pandas
import itertools

mp.rcParams.update({'mathtext.default': 'regular'})
% matplotlib inline

In [2]:
working_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/'
#save_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calculations/npy_files/'
#latlon_indices = numpy.load(working_dir + 'ccal_latlon_indices_array.npy'); region='ccal'
#latlon_indices = numpy.load(working_dir + 'ncal_latlon_indices_array.npy'); region='ncal'
#latlon_indices = numpy.load(working_dir + 'scal_latlon_indices_array.npy'); region='scal'

PRECT_lat = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lat.npy')
PRECT_lon = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lon.npy')

In [3]:
working_dir = '/Users/baird/google_drive/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/'
#threshold=0.0
threshold=0.1
#threshold=0.5
#threshold=1.0
#threshold=5.0
#threshold=10.0

In [4]:
PRECT_nlat = 26
PRECT_nlon = 25

latlon_indices = list(itertools.product(range(PRECT_nlat), range(PRECT_nlon)))
region = 'whole_domain'
window=1

# Open preindustrial control data

In [5]:
year_start_pic = 402 #time_subsets[chunk,0]
year_end_pic = 2200 #time_subsets[chunk,1]

# create season strings
years_pic = numpy.arange(year_start_pic, year_end_pic+1, 1).astype(numpy.int)
half_years_pic = numpy.arange(year_start_pic+0.75, year_end_pic, 1)
#season_strings_pic = numpy.empty(years.size-1, dtype=numpy.str)

season_strings_pic = [str(years_pic[i])+'-'+str(years_pic[i+1]) for i in range(years_pic.size-1)]
member_strings_pic = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_pic=year_end_pic-year_start_pic

In [9]:
latlon_idx = 200
working_dir = '/Users/baird/Dropbox/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/whole_domain/'
filename = 'member_005_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_'+'{:04d}'.format(year_start_pic)+'-'+'{:04d}'.format(year_end_pic)+'_threshold_'+str(threshold)+'mmday_'+region+'.npy'
dict_pic = numpy.load(working_dir + filename).item()

In [10]:
dict_pic['402-403']

{'monthly_totals': [156.48085904121399,
  188.29112079739571,
  318.29036964476109,
  318.87784793227911,
  211.44120836257935],
 'running_40d_sum': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  181.38755264878273,
  181.11891874670982,
  178.02706894278526,
  149.99140760302544,
  170.21745654940605,
  170.54137381911278,
  161.99959513545036,
  161.29031643271446,
  156.3791022002697,
  151.6867319047451,
  152.03108605742455,
  153.08924940228462,
  137.80511239171028,
  127.25107815861702,
  141.73346856236458,
  156.44486001133919,
  173.19499638676643,
  169.67128339409828,
  189.03661218285561,
  193.4841305911541,
  194.99890229105949,
  203.03898903727531,
  202.79399517178535,
  206.08966115117073,
  204.41693890094757,
  194.49607738852501,
  194.7968

# Contruct extreme value distributions for each grid point

# Open historical and RCP8.5 data

In [10]:
year_start = 1920
year_end = 2100

year_start_list = [1920,1950,1980,2010,2040,2070]
year_end_list = [1950,1980,2010,2040,2070,2100]

# create season strings
years = numpy.arange(year_start, year_end+1, 1).astype(numpy.int)
half_years_hist_rcp = numpy.arange(year_start+0.75, year_end, 1)

season_strings_hist_rcp = [str(years[i])+'-'+str(years[i+1]) for i in range(years.size-1)]
member_strings_hist_rcp = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_hist_rcp=year_end-year_start

In [13]:
ensemble_members = numpy.hstack((numpy.arange(1,36), numpy.arange(101,106)))
ensemble_names = ['{:03d}'.format(i) for i in ensemble_members]

In [ ]:
working_dir = '/Users/baird/Dropbox/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/whole_domain/'
# for each grid point:
# cycle through all times, get extreme seas_accum sum in each time period, store in array with length(season_strings_pic)
# once this is done, calculate the return periods of each of these
extreme_values_seas_accum = numpy.zeros(( PRECT_nlat*PRECT_nlon, len(season_strings_hist_rcp) ))

for ensemble_idx in range(len(ensemble_members)):
    print(ensemble_names[ensemble_idx])
    for latlon_idx in range(len(latlon_indices)):
        filename = 'member_'+ensemble_names[ensemble_idx]+'_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_1920-2100_threshold_'+str(threshold)+'mmday_'+region+'.npy'
        #if latlon_idx%10==0:
        #    print(latlon_idx)
        dict_hist_rcp = numpy.load(working_dir + filename).item()
        sum_seas_accum_list = [dict_hist_rcp[i]['running_seas_accum'] for i in dict_hist_rcp.keys()]
        for season_idx in range(len(season_strings_hist_rcp)):
            extreme_values_seas_accum[latlon_idx, season_idx] = numpy.nanmax(sum_seas_accum_list[season_idx])

    extreme_values_seas_accum_df = pandas.DataFrame(extreme_values_seas_accum, columns=[season_strings_hist_rcp])
    csv_filename = 'extreme_values_seas_accums_dataframe_model_'+ensemble_names[ensemble_idx]+'_1920-2100.csv'
    extreme_values_seas_accum_df.to_csv('csv_files/'+csv_filename)
    print(csv_filename, 'written!')

001
extreme_values_40d_sums_dataframe_model_001_1920-2100.csv written!
002
extreme_values_40d_sums_dataframe_model_002_1920-2100.csv written!
003
extreme_values_40d_sums_dataframe_model_003_1920-2100.csv written!
004
extreme_values_40d_sums_dataframe_model_004_1920-2100.csv written!
005
extreme_values_40d_sums_dataframe_model_005_1920-2100.csv written!
006
extreme_values_40d_sums_dataframe_model_006_1920-2100.csv written!
007
extreme_values_40d_sums_dataframe_model_007_1920-2100.csv written!
008
